In [6]:
#Example of how to get Landsat data using the getImagesLib and view outputs using the Python visualization tools
#Acquires Landsat data and then adds them to the viewer
####################################################################################################
#Module imports
# !python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map
Map.port = 1234

# Enter proxy url (see in url bar- e.g. https://someCode-dot-someRegion.notebooks.googleusercontent.com/)
# If left as None, will prompt user at first Map.view() call 
Map.proxy_url = 'https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/'

In [7]:
#This example will use all default parameters to demonstrate how to use the basic composite functionality

#First clear the map in case it has been populated with layers/commands earlier
Map.clearMap()
#Define user parameters:

# Specify study area: Study area
# Can be a featureCollection, feature, or geometry
studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile').geometry().bounds()#testAreas['CA']
studyArea = ee.Geometry.Polygon(
        [[[-66.29465453845745, 18.491553939984392],
          [-66.29465453845745, 18.144770192006572],
          [-65.58054321033245, 18.144770192006572],
          [-65.58054321033245, 18.491553939984392]]], None, False)
# Update the startJulian and endJulian variables to indicate your seasonal
# constraints. This supports wrapping for tropics and southern hemisphere.
# If using wrapping and the majority of the days occur in the second year, the system:time_start will default
# to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
# startJulian: Starting Julian date
# endJulian: Ending Julian date
startJulian = 153
endJulian = 152

# Specify start and end years for all analyses
# More than a 3 year span should be provided for time series methods to work
# well. If providing pre-computed stats for cloudScore and TDOM, this does not
# matter
startYear = 2009
endYear = 2011

#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea,startYear,endYear,startJulian,endJulian)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']



Map.clearMap()
# Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',False)
# for year in range(startYear,endYear + 1 ):
    #  t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'Default Params {}-{}'.format(startJulian,endJulian),'True')

Map.centerObject(ee.Geometry.Polygon(
        [[[-65.8337045819611, 18.329538797654042],
          [-65.8337045819611, 18.235653085671174],
          [-65.70461522649235, 18.235653085671174],
          [-65.70461522649235, 18.329538797654042]]], None, False))
Map.turnOnInspector()
Map.view()



Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jun 02 2009 , End date: May 31 2012
startYear :  2009
endYear :  2011
startJulian :  153
endJulian :  152
toaOrSR :  TOA
includeSLCOffL7 :  False
defringeL5 :  False
applyQABand :  False
applyCloudProbability :  True
applyShadowShift :  False
applyCloudScoreLandsat :  False
applyCloudScoreSentinel2 :  False
applyTDOMLandsat :  True
applyTDOMSentinel2 :  True
applyFmaskCloudMask :  True
applyFmaskCloudShadowMask :  True
applyFmaskSnowMask :  False
cloudScoreThresh :  20
performCloudScoreOffset :  True
cloudScorePctl :  10
zScoreThresh :  -1
shadowSumBands :  ['nir', 'swir1']
landsatResampleMethod :  near
sentinel2ResampleMethod :  aggregate
convertToDailyMosaics :  True
runChastainHarmonization :  True
correctIllumination :  False
correctScale :  250
preComputedLandsatCloudScoreOffset :  None
preComputedLandsatTDOMIRMean :  None
preComputedLandsatTDOMIRStdDev :  None
preComputedSentinel2CloudScoreOffset :  None
preComputedSentinel

127.0.0.1 - - [28/Jul/2023 17:34:01] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2023 17:34:10] "GET /geeView/images/cumulative_icon.png HTTP/1.1" 200 -


In [ ]:
#It is clear the default parameters do not work very well in this area
#There are missing data and cloud artifacts
#You can access the parameters that were used through the properties of the returned collection
print(processedComposites.toDictionary().getInfo())
print(processedScenes.toDictionary().getInfo())



In [3]:
#Since there are not that many images available in this area for these years, let's try adding Landsat 7
includeSLCOffL7 = True
#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea,startYear,endYear,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#Turn off layers from previous iteration
Map.turnOffAllLayers()

# Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',False)
# for year in range(startYear,endYear + 1 ):
    #  t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 added {}-{}'.format(startJulian,endJulian),True)


Map.view()

#You'll notice this helps fill in the holes, but introduces many cloud-related artifacts

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jun 02 2009 , End date: May 31 2012
startYear :  2009
endYear :  2011
startJulian :  153
endJulian :  152
toaOrSR :  TOA
includeSLCOffL7 :  True
defringeL5 :  False
applyQABand :  False
applyCloudProbability :  True
applyShadowShift :  False
applyCloudScoreLandsat :  False
applyCloudScoreSentinel2 :  False
applyTDOMLandsat :  True
applyTDOMSentinel2 :  True
applyFmaskCloudMask :  True
applyFmaskCloudShadowMask :  True
applyFmaskSnowMask :  False
cloudScoreThresh :  20
performCloudScoreOffset :  True
cloudScorePctl :  10
zScoreThresh :  -1
shadowSumBands :  ['nir', 'swir1']
landsatResampleMethod :  near
sentinel2ResampleMethod :  aggregate
convertToDailyMosaics :  True
runChastainHarmonization :  True
correctIllumination :  False
correctScale :  250
preComputedLandsatCloudScoreOffset :  None
preComputedLandsatTDOMIRMean :  None
preComputedLandsatTDOMIRStdDev :  None
preComputedSentinel2CloudScoreOffset :  None
preComputedSentinel2

127.0.0.1 - - [28/Jul/2023 17:30:53] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2023 17:30:58] "GET /geeView/images/cumulative_icon.png HTTP/1.1" 200 -


In [ ]:
#Let's try to improve the cloud masking. Fmask is used by default, but misses some clouds
#We'll try the adding in the cloudScore method
applyCloudScore = True

#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea,startYear,endYear,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7,applyCloudScoreLandsat=applyCloudScore)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#Turn off layers from previous iteration
Map.turnOffAllLayers()


Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 and CloudScore added {}-{}'.format(startJulian,endJulian),True)


Map.view()

#You'll notice this cleans up the cloud masking a lot

In [ ]:
#You'll still notice there are some dark areas likely due to cloud shadow masking omission
#Fmasks's cloud shadow mask misses a lot typically. A temporal outlier method called the
#Temporal Dark Outlier Mask (TDOM) works well with masking cloud shadows

#We'll try the cloudScore method
applyTDOM = True

#Call on master wrapper function to get Landat scenes and composites
#In order to identify dark outliers, we will extend the dates by 6 years to get a larger sample
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea,startYear,endYear,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7,applyCloudScoreLandsat=applyCloudScore)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'CloudScore and TDOM added {}-{}'.format(startJulian,endJulian),True)


Map.view()

#You'll notice this cleans up the cloud masking a lot

In [ ]:
# Now that we've gone through some of the parameters that can be changed for Landsat composites, we will use both Landsat and Sentinel2 to get the best possible composites

#Provide location composites will be exported to
#This should be an asset folder, or more ideally, an asset imageCollection
exportPathRoot = 'projects/rcr-gee/assets/composites-lcms-training-module-1'
# Make sure collection exists
aml.create_image_collection(exportPathRoot)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(exportPathRoot,writers = [],all_users_can_read = True,readers = [])

lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(\
  studyArea = studyArea,
  startYear = 2010,
  endYear = 2022,
  startJulian = 273,
  endJulian = 272,
  timebuffer =  0,
  weights =  [1],
  compositingMethod = 'medoid',
  toaOrSR = 'TOA',
  includeSLCOffL7 = True,
  defringeL5 = True,
  applyQABand = False,
  applyCloudProbability = True,
  applyShadowShift = False,
  applyCloudScoreLandsat = True,
  applyCloudScoreSentinel2 = False,
  applyTDOMLandsat = True,
  applyTDOMSentinel2 = True,
  applyFmaskCloudMask = True,
  applyFmaskCloudShadowMask = True,
  applyFmaskSnowMask = False,
  cloudHeights = ee.List.sequence(500,10000,500),
  cloudScoreThresh = 20,
  performCloudScoreOffset = True,
  cloudScorePctl = 10,
  zScoreThresh = -1,
  shadowSumThresh = 0.35,
  contractPixels = 1.5,
  dilatePixels = 3.5,
  shadowSumBands = ['nir','swir1'],
  landsatResampleMethod = 'near',
  sentinel2ResampleMethod = 'aggregate',
  convertToDailyMosaics = True,
  runChastainHarmonization = False,# Set to True in order to perform regression harmonization between Landsat and Sentinel 2
  correctIllumination = False,
  correctScale = 250,
  exportComposites = True,
  outputName = 'Landsat-Sentinel2-Hybrid',
  exportPathRoot = exportPathRoot,
  crs = 'EPSG:5070',
  transform = [30,0,-2361915.0,0,-30,3177735.0],
  scale = None,
  preComputedLandsatCloudScoreOffset = None,
  preComputedLandsatTDOMIRMean = None,
  preComputedLandsatTDOMIRStdDev = None,
  preComputedSentinel2CloudScoreOffset = None,
  preComputedSentinel2TDOMIRMean = None,
  preComputedSentinel2TDOMIRStdDev = None,
  cloudProbThresh = 40,
  landsatCollectionVersion = 'C2',
  overwrite = False)
#Call on master wrapper function to get Landat scenes and composites
#In order to identify dark outliers, we will extend the dates by 6 years to get a larger sample
# lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea,startYear-3,endYear+3,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7,applyCloudScoreLandsat=applyCloudScore,applyTDOMLandsat=applyTDOM)
# lsAndTs = getImagesLib.getLandsatWrapper(studyArea,startYear-3,endYear+3,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7,applyCloudScore=applyCloudScore,applyTDOM=applyTDOM)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#Turn off layers from previous iteration
Map.turnOffAllLayers()

# Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',False)
# for year in range(startYear,endYear + 1 ):
    #  t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'Landsat and S2 CloudScore and TDOM added {}-{}'.format(startJulian,endJulian),True)


Map.view()

tml.trackTasks2()
#You'll notice this cleans up the cloud masking a lot

In [ ]:
# View composites as they are completed
composites = ee.ImageCollection(exportPathRoot)
Map.clearMap()

# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse10k,'Composites')

# By looking at the Sensor that is used, you can see with the introduction of Landsat 8 in 2013 and then Sentinel 2 in 2016-2017, composites become much better quality
Map.addTimeLapse(composites.select('sensor'),{'min':4,'max':22,'palette':'088,808','classLegendDict':{'Landsat (Landsat 5 = 5, Landsat 7 = 7, etc...)':'088','Sentinel2 (Sentinel 2A = 21, Sentinel 2B = 22, etc....)':'808'}},'Sensor')
Map.addTimeLapse(composites.select('compositeObsCount'),{'min':3,'max':10,'palette':'D0D,0D0'},'Composite Observation Counts')
Map.turnOnInspector()
Map.view()
# Now that we have exported composites, we will use them in the LandTrendr temporal segmentation algorithm

> There are many different parameters that can be changed in order to improve composites in different study areas
> This is just one example. Other parameters include changing date ranges, and reducers